<a href="https://colab.research.google.com/github/leejh12345/ai/blob/main/crab_age_prediction/2024_AI_Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

kaggle에서 baseline 코드가 안보이는 관계로 부득이하게 colab에서 실행   

files.upload()까지 코드가 진행되면, kaggle login창이 나타나고 로그인 진행   
이전에 kaggle 사이트에서 프로필 사진 클릭 후 -> settings 진입 -> create new token
여기서 생성된 kaggle.json을 업로드
그러면 private 데이터라고 하더라도 현재 colab으로 들고올 수 있음


In [25]:
!pip install kaggle

from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c 2024-csu-iceai-crab-age-regression-competition

!mkdir -p /kaggle/input/2024-csu-iceai-crab-age-regression-competition
!mkdir -p /kaggle/output/2024-csu-iceai-crab-age-regression-competition
!unzip 2024-csu-iceai-crab-age-regression-competition.zip -d /kaggle/input/2024-csu-iceai-crab-age-regression-competition

Saving sample_submission.csv to sample_submission (1).csv
Saving test_v2.csv to test_v2 (1).csv
Saving train.csv to train (1).csv
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 7, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 407, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/
unzip:  cannot find or open 2024-csu-iceai-crab-age-regression-competition.zip, 2024-csu-iceai-crab-age-regression-competition.zip.zip or 2024-csu-iceai-crab

## 환경 설정   
라이브러리 설치   
pandas, scikt-learn

In [26]:
!pip install pandas scikit-learn

## 0. 필요한 라이브러리 호출

In [27]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [28]:
!ls /kaggle/input/2024-csu-iceai-crab-age-regression-competition

sample_submission.csv  test_v2.csv  train.csv


## 1-1. 데이터셋 로드

In [29]:
data = pd.read_csv('/kaggle/input/2024-csu-iceai-crab-age-regression-competition/train.csv')
test_data = pd.read_csv('/kaggle/input/2024-csu-iceai-crab-age-regression-competition/test_v2.csv')

print(data.head())

   id Sex  Length  Diameter  Height     Weight  Shucked Weight  \
0   0   M  1.6375     1.300  0.4750  38.711242       17.690088   
1   1   M  1.5375     1.175  0.3750  33.409886       11.495722   
2   2   F  1.7125     1.375  0.4750  43.629880       16.726205   
3   3   F  1.6375     1.275  0.4375  38.980562       16.683681   
4   4   F  1.7125     1.375  0.4250  51.511041       22.254358   

   Viscera Weight  Shell Weight   Age  
0        7.413394     11.339800  10.0  
1        8.022908      8.930093  17.0  
2       11.424848     14.033003  13.0  
3        6.676307     11.580771  11.0  
4       11.439023     12.332033  10.0  


## 1-2. 데이터셋 전처리

In [30]:
# 데이터 로드
data = pd.read_csv('/kaggle/input/2024-csu-iceai-crab-age-regression-competition/train.csv')
test_data = pd.read_csv('/kaggle/input/2024-csu-iceai-crab-age-regression-competition/test_v2.csv')

# 데이터 전처리 추가
from sklearn.preprocessing import LabelEncoder

# Sex 컬럼 인코딩
le = LabelEncoder()
data['Sex'] = le.fit_transform(data['Sex'])
test_data['Sex'] = le.fit_transform(test_data['Sex'])

# 추가: 이상치 제거 (예: Weight 열)
Q1 = data['Weight'].quantile(0.25)
Q3 = data['Weight'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
data = data[(data['Weight'] >= lower_bound) & (data['Weight'] <= upper_bound)]


# 추가 변수 생성
data['Weight_to_Length'] = data['Weight'] / data['Length']
data['Shucked_to_Weight'] = data['Shucked Weight'] / data['Weight']
data['Viscera_to_Weight'] = data['Viscera Weight'] / data['Weight']
data['Length_to_Height'] = data['Length'] / (data['Height'] + 1e-8)

test_data['Weight_to_Length'] = test_data['Weight'] / test_data['Length']
test_data['Shucked_to_Weight'] = test_data['Shucked Weight'] / test_data['Weight']
test_data['Viscera_to_Weight'] = test_data['Viscera Weight'] / test_data['Weight']
test_data['Length_to_Height'] = test_data['Length'] / (test_data['Height'] + 1e-8)

# 추가: 'Volume' 및 'Density'
data['Volume'] = data['Length'] * data['Diameter'] * data['Height']
data['Density'] = data['Weight'] / data['Volume']

test_data['Volume'] = test_data['Length'] * test_data['Diameter'] * test_data['Height']
test_data['Density'] = test_data['Weight'] / test_data['Volume']

# 추가: 'Length*Diameter' 및 'Weight/Height'
data['Length*Diameter'] = data['Length'] * data['Diameter']
data['Weight/Height'] = data['Weight'] / (data['Height'] + 1e-8)

test_data['Length*Diameter'] = test_data['Length'] * test_data['Diameter']
test_data['Weight/Height'] = test_data['Weight'] / (test_data['Height'] + 1e-8)



# 추가: 로그 변환
data['Log_Weight'] = np.log1p(data['Weight'])
test_data['Log_Weight'] = np.log1p(test_data['Weight'])



In [31]:
X = data[['Sex', 'Length', 'Diameter', 'Height', 'Weight',
          'Shucked Weight', 'Viscera Weight', 'Shell Weight',
          'Volume', 'Density', 'Weight_to_Length',
          'Shucked_to_Weight', 'Viscera_to_Weight', 'Length_to_Height',
          'Length*Diameter', 'Weight/Height', 'Log_Weight']]

y = data['Age']

X_test = test_data[['Sex', 'Length', 'Diameter', 'Height', 'Weight',
                    'Shucked Weight', 'Viscera Weight', 'Shell Weight',
                    'Volume', 'Density', 'Weight_to_Length',
                    'Shucked_to_Weight', 'Viscera_to_Weight', 'Length_to_Height',
                    'Length*Diameter', 'Weight/Height', 'Log_Weight']]

# 데이터셋 분리
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# NaN 값 처리
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_valid.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)

X_train.fillna(X_train.mean(), inplace=True)
X_valid.fillna(X_valid.mean(), inplace=True)
X_test.fillna(X_test.mean(), inplace=True)

# 스케일링
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

<ipython-input-31-6a791e877256>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
<ipython-input-31-6a791e877256>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.fillna(X_test.mean(), inplace=True)


## 3.a. Decision tree를 이용한 regression model

In [32]:
dt_regressor = DecisionTreeRegressor(random_state = 42)
dt_regressor.fit(X_train, y_train)
y_pred_dt = dt_regressor.predict(X_valid)

## 3.b. support vector machine를 이용한 regression model

In [33]:
from sklearn.model_selection import RandomizedSearchCV

svm_params = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf']
}

svm_search = RandomizedSearchCV(
    SVR(),
    param_distributions=svm_params,
    n_iter=10, scoring='neg_mean_absolute_error', cv=3, random_state=42
)
svm_search.fit(X_train_scaled, y_train)
svm_regressor = svm_search.best_estimator_

y_pred_svm = svm_regressor.predict(X_valid_scaled)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


In [34]:
# 추가: Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error

rf_params = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_search = RandomizedSearchCV(
    RandomForestRegressor(random_state=42),
    param_distributions=rf_params,
    n_iter=10, scoring='neg_mean_absolute_error', cv=3, random_state=42
)
rf_search.fit(X_train, y_train)
rf_regressor = rf_search.best_estimator_

y_pred_rf = rf_regressor.predict(X_valid)


In [35]:
# Additional Model: XGBoost Regressor
from xgboost import XGBRegressor

xgb_params = {
    'n_estimators': [100, 200, 500],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'subsample': [0.7, 0.8, 1.0],
    'colsample_bytree': [0.7, 0.8, 1.0]
}

xgb_search = RandomizedSearchCV(
    XGBRegressor(random_state=42),
    param_distributions=xgb_params,
    n_iter=20, scoring='neg_mean_absolute_error', cv=3, random_state=42
)
xgb_search.fit(X_train_scaled, y_train)
xgb_regressor = xgb_search.best_estimator_

y_pred_xgb = xgb_regressor.predict(X_valid_scaled)

In [36]:
lgb_params = {
    'n_estimators': [100, 200, 500],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 4, 5],  # 기존 -1에서 유효한 값으로 수정
    'num_leaves': [15, 31, 50],  # 리프 노드 제한 추가
    'min_data_in_leaf': [10, 20, 30],  # 최소 샘플 수 추가
    'subsample': [0.7, 0.8, 1.0]
}

lgb_search = RandomizedSearchCV(
    LGBMRegressor(random_state=42),
    param_distributions=lgb_params,
    n_iter=10, scoring='neg_mean_absolute_error', cv=3, random_state=42
)

# 모델 학습
lgb_search.fit(X_train_scaled, y_train)
lgb_regressor = lgb_search.best_estimator_

# 검증 데이터 예측
y_pred_lgb = lgb_regressor.predict(X_valid_scaled)


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

In [39]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input, Attention, Concatenate, Reshape

# DNN 모델 정의
def build_dnn_model(input_shape):
    model = Sequential([
        Dense(128, activation='relu', input_shape=(input_shape,)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(1)  # 최종 출력층
    ])
    model.compile(optimizer='adam', loss='mae', metrics=['mae'])
    return model

# DNN+Attention 모델 정의
def build_dnn_attention_model(input_shape):
    input_layer = Input(shape=(input_shape,))

    # Dense Layers
    dnn = Dense(128, activation='relu')(input_layer)
    dnn = Dense(64, activation='relu')(dnn)

    # 차원 확장 (Attention Layer 사용을 위한 3D 입력 생성)
    dnn_expanded = Reshape((1, 64))(dnn)

    # Attention Layer
    attention_output = Attention()([dnn_expanded, dnn_expanded])

    # Attention 출력 차원 축소 (3D -> 2D)
    attention_output_flat = Reshape((64,))(attention_output)

    # Concatenate Attention output with Dense layers
    combined = Concatenate()([dnn, attention_output_flat])
    output = Dense(1)(combined)

    model = Model(inputs=input_layer, outputs=output)
    model.compile(optimizer='adam', loss='mae', metrics=['mae'])
    return model

# DNN 모델 학습
dnn_model = build_dnn_model(X_train_scaled.shape[1])
dnn_model.fit(X_train_scaled, y_train, validation_data=(X_valid_scaled, y_valid), epochs=50, batch_size=32, verbose=1)

# DNN+Attention 모델 학습
dnn_attention_model = build_dnn_attention_model(X_train_scaled.shape[1])
dnn_attention_model.fit(X_train_scaled, y_train, validation_data=(X_valid_scaled, y_valid), epochs=50, batch_size=32, verbose=1)


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


374/374 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 3.6991 - mae: 3.6991 - val_loss: 1.5462 - val_mae: 1.5462
Epoch 2/50
374/374 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.5259 - mae: 1.5259 - val_loss: 1.4675 - val_mae: 1.4675
Epoch 3/50
374/374 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.4740 - mae: 1.4740 - val_loss: 1.3893 - val_mae: 1.3893
Epoch 4/50
374/374 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.3980 - mae: 1.3980 - val_loss: 1.3718 - val_mae: 1.3718
Epoch 5/50
374/374 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.3899 - mae: 1.3899 - val_loss: 1.3514 - val_mae: 1.3514
Epoch 6/50
374/374 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.3631 - mae: 1.3631 - val_loss: 1.3539 - val_mae: 1.3539
Epoch 7/50
374/374 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 1.3680 - mae: 1.3680 - val_loss: 1.3359 - val_mae: 1.3359
Epoch 8/50
374/374 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.3587 - mae: 1.3587 - val_loss: 1.3376 - val_mae: 1.3376
Epoch 9/50
374/374 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.3373 

/usr/local/lib/python3.10/dist-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


374/374 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4.1438 - mae: 4.1438

/usr/local/lib/python3.10/dist-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


374/374 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 4.1391 - mae: 4.1391 - val_loss: 1.5507 - val_mae: 1.5507
Epoch 2/50
374/374 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 1.5208 - mae: 1.5208 - val_loss: 1.4601 - val_mae: 1.4601
Epoch 3/50
374/374 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.4448 - mae: 1.4448 - val_loss: 1.4407 - val_mae: 1.4407
Epoch 4/50
374/374 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.4037 - mae: 1.4037 - val_loss: 1.3864 - val_mae: 1.3864
Epoch 5/50
374/374 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.4010 - mae: 1.4010 - val_loss: 1.3742 - val_mae: 1.3742
Epoch 6/50
374/374 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.3795 - mae: 1.3795 - val_loss: 1.3853 - val_mae: 1.3853
Epoch 7/50
374/374 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.3626 - mae: 1.3626 - val_loss: 1.4026 - val_mae: 1.4026
Epoch 8/50
374/374 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.3485 - mae: 1.3485 - val_loss: 1.3486 - val_mae: 1.3486
Epoch 9/50
374/374 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.3631 

## 4. 검증 데이터를 이용한 평가

In [41]:
# DNN 및 DNN+Attention 예측값 생성
y_pred_dnn = dnn_model.predict(X_valid_scaled)
y_pred_dnn_attention = dnn_attention_model.predict(X_valid_scaled)

# MAE 평가
mae_dt = mean_absolute_error(y_valid, y_pred_dt)
mae_svm = mean_absolute_error(y_valid, y_pred_svm)
mae_rf = mean_absolute_error(y_valid, y_pred_rf)
mae_xgb = mean_absolute_error(y_valid, y_pred_xgb)
mae_lgb = mean_absolute_error(y_valid, y_pred_lgb)
mae_dnn = mean_absolute_error(y_valid, y_pred_dnn)
mae_dnn_attention = mean_absolute_error(y_valid, y_pred_dnn_attention)

print(f"Decision Tree MAE: {mae_dt:.4f}")
print(f"SVM MAE: {mae_svm:.4f}")
print(f"Random Forest MAE: {mae_rf:.4f}")
print(f"XGBoost MAE: {mae_xgb:.4f}")
print(f"LightGBM MAE: {mae_lgb:.4f}")
print(f"DNN MAE: {mae_dnn:.4f}")
print(f"DNN+Attention MAE: {mae_dnn_attention:.4f}")

# 스택 앙상블 학습 데이터 생성
stacked_train = pd.DataFrame({
    'rf': y_pred_rf,
    'xgb': y_pred_xgb,
    'lgb': y_pred_lgb,
    'dnn': y_pred_dnn.flatten(),
    'dnn_attention': y_pred_dnn_attention.flatten()
})

# Ridge 모델 학습
from sklearn.linear_model import Ridge
ridge = Ridge()
ridge.fit(stacked_train, y_valid)

# DNN 및 DNN+Attention 테스트 데이터 예측값 생성
y_pred_dnn_test = dnn_model.predict(X_test_scaled)
y_pred_dnn_attention_test = dnn_attention_model.predict(X_test_scaled)

# 스택 앙상블 테스트 데이터 생성
stacked_test = pd.DataFrame({
    'rf': rf_regressor.predict(X_test_scaled),
    'xgb': xgb_regressor.predict(X_test_scaled),
    'lgb': lgb_regressor.predict(X_test_scaled),
    'dnn': y_pred_dnn_test.flatten(),
    'dnn_attention': y_pred_dnn_attention_test.flatten()
})

# 최종 예측값 생성
final_predictions = ridge.predict(stacked_test)

# 결과 정규화 (범위: 1~25)
final_predictions = np.clip(final_predictions, 1, 25)



94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (32, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


86/94 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step

/usr/local/lib/python3.10/dist-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step
Decision Tree MAE: 1.9163
SVM MAE: 1.3210
Random Forest MAE: 1.3564
XGBoost MAE: 1.3310
LightGBM MAE: 1.3335
DNN MAE: 1.3267
DNN+Attention MAE: 1.3235
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20


## 5. 제출 파일 생성

In [44]:
# 가중치 기반 앙상블
weighted_predictions_train = (
    w_dt * dt_regressor.predict(X_valid_scaled) +
    w_svm * svm_regressor.predict(X_valid_scaled) +
    w_rf * rf_regressor.predict(X_valid_scaled) +
    w_xgb * xgb_regressor.predict(X_valid_scaled) +
    w_lgb * lgb_regressor.predict(X_valid_scaled) +
    w_dnn * y_pred_dnn.flatten() +
    w_dnn_attention * y_pred_dnn_attention.flatten()
)

# 결과 정규화 (훈련 데이터)
weighted_predictions_train = np.clip(weighted_predictions_train, 1, 25)

# 스택 앙상블 학습 데이터 생성 (weighted 추가)
stacked_train = pd.DataFrame({
    'rf': y_pred_rf,
    'xgb': y_pred_xgb,
    'lgb': y_pred_lgb,
    'dnn': y_pred_dnn.flatten(),
    'dnn_attention': y_pred_dnn_attention.flatten(),
    'weighted': weighted_predictions_train  # weighted 추가
})

# Ridge 모델 학습
ridge = Ridge()
ridge.fit(stacked_train, y_valid)

# 스택 앙상블 테스트 데이터 생성 (weighted 추가)
stacked_test = pd.DataFrame({
    'rf': rf_regressor.predict(X_test_scaled),
    'xgb': xgb_regressor.predict(X_test_scaled),
    'lgb': lgb_regressor.predict(X_test_scaled),
    'dnn': y_pred_dnn_test.flatten(),
    'dnn_attention': y_pred_dnn_attention_test.flatten(),
    'weighted': weighted_predictions  # weighted 추가
})

# 스택 앙상블을 사용한 최종 예측
final_predictions = ridge.predict(stacked_test)

# 최종 예측 정규화 (범위: 1~25)
final_predictions = np.clip(final_predictions, 1, 25)

# 제출 파일 생성
results = pd.DataFrame({
    'id': range(len(final_predictions)),
    'Age': final_predictions
})
results.to_csv('final_submission.csv', index=False)

from google.colab import files
files.download('final_submission.csv')



/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>